KÜTÜPHANE EKLEME

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller, kpss
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.api import SARIMAX, api as smt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import OneClassSVM, SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import mutual_info_regression
from xgboost import XGBRegressor

KORELASYON VE ÖZELLİK HESAPLAMA

In [ ]:
file_path = 'organised_data_final.xlsx'
data = pd.read_excel(file_path)

correlation_with_OUP = data.corr()['OUP'].drop('OUP')  # 'OUP' hariç diğer değişkenlerle korelasyon

plt.figure(figsize=(10, 6))
bars = correlation_with_OUP.plot(kind='bar', color='skyblue')
plt.xlabel('Değişkenler')
plt.ylabel('Korelasyon')
plt.title('OUP Değişkeninin Diğer Değişkenlerle Korelasyonu')
plt.xticks(rotation=45)
plt.tight_layout()

for bar in bars.patches:
    plt.text(bar.get_x() + bar.get_width() / 2 - 0.1, bar.get_height() + 0.01, f'{bar.get_height():.2f}', ha='center', va='bottom', color='black', fontsize=8)

plt.show()

#####

# Veri setinin temel istatistiksel özetini almak
summary_stats = data.describe()
summary_stats.to_excel('summary_stats.xlsx')

AYKIRI DEĞERLER

In [ ]:
dosya_yolu = 'organised_data_final.xlsx'
veri = pd.read_excel(dosya_yolu)

veri = veri.dropna()

# Modified Z-Score hesaplama
mad = median_abs_deviation(veri)
modified_z_scores = 0.6745 * (veri - veri.median()) / mad

# Aykırı değerleri saptama ve işaretleme
threshold = 3.5  # Eşik değeri
aykiri_degerler = (abs(modified_z_scores) > threshold).any(axis=1)

# Isolation Forest kullanarak aykırı değerleri saptama
model = IsolationForest(contamination=0.1)  # Kontaminasyon oranı
model.fit(veri)
isolation_outliers = model.predict(veri)

# Aykırı değerleri görselleştirme
plt.figure(figsize=(10, 6))

plt.subplot(2, 1, 1)
plt.scatter(veri.index, veri['OUP'], c=aykiri_degerler, cmap='coolwarm')
plt.title('Modified Z-Score ile Aykırı Değerler')

plt.subplot(2, 1, 2)
plt.scatter(veri.index, veri['OUP'], c=isolation_outliers, cmap='coolwarm')
plt.title('Isolation Forest ile Aykırı Değerler')

plt.tight_layout()
plt.show()

ÖNEMLİLİK SKORLARI

In [ ]:
df = pd.read_excel('organised_data_final.xlsx')

df_clean = df.dropna()

X = df_clean[['WDS', 'MXT', 'TMP', 'MNT', 'PRC', 'PRS', 'RLH', 'SSI', 'GSR']]
y = df_clean['OUP']

X.columns = ['WDS', 'MXT', 'TMP', 'MNT', 'PRC', 'PRS', 'RLH', 'SSI', 'GSR']

model = RandomForestRegressor()
model.fit(X, y)

# Özellik önemlilik hesaplama
feature_importance = model.feature_importances_

# Özellik önemlilik skorlarını saklamak için bir DataFrame oluşturun
feature_importance_df = pd.DataFrame({'Özellik': X.columns, 'Önemlilik': feature_importance})

# Özellikleri önemlilik skorlarına göre azalan sırada sıralayın
feature_importance_df = feature_importance_df.sort_values(by='Önemlilik', ascending=False)

# Özellik önemliliklerini görselleştirme
plt.figure(figsize=(10, 6))
plt.barh(feature_importance_df['Özellik'], feature_importance_df['Önemlilik'], color='skyblue')
plt.xlabel('Özellik Önemlilik Skoru')
plt.title('Özellik Önemlilik Skorları')  # En önemli özelliğin üstte olması için y-eksenini ters çevirme
plt.show()

DOĞRULAMA SKORU

In [ ]:
file_path = 'organised_data_final.xlsx'
data = pd.read_excel(file_path)

columns_with_missing_values = ['WDS','MXT', 'MNT', 'TMP', 'PRC', 'PRS', 'RLH', 'SSI', 'GSR', 'OUP']

rows_with_missing_values = data[columns_with_missing_values].isnull().any(axis=1)

data_with_missing_values = data[rows_with_missing_values]

data_without_missing_values = data.drop(columns_with_missing_values, axis=1)

imputer = SimpleImputer(strategy='mean')
imputed_data = imputer.fit_transform(data_without_missing_values)

model = XGBRegressor()

scores = cross_val_score(model, imputed_data[~rows_with_missing_values], data[columns_with_missing_values][~rows_with_missing_values], scoring='neg_mean_squared_error', cv=5)

print("Çapraz doğrulama skorları: ", scores)
print("Ortalama çapraz doğrulama skoru: ", scores.mean())

# Skorları grafiğe dökme
plt.figure(figsize=(8, 6))
plt.bar(range(1, len(scores)+1), -scores, color='skyblue')
plt.xlabel('Çapraz Doğrulama Parçası')
plt.ylabel('Negatif MSE Skoru')
plt.title('XGBoost Regressor Çapraz Doğrulama Skorları')
plt.ylim(min(-scores)*0.9, max(-scores)*1.1)  # Grafik aralığını düzenleme
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

KARŞILIKLI BİLGİ DEĞERLERİ

In [ ]:
file_path = 'organised_data_final.xlsx'
ata = pd.read_excel(file_path)
data = data.dropna()

# Özellikler ve hedef değişkeni seçme
features = data.drop(['YEAR', 'MONTH', 'DAY', 'HOUR', 'OUP'], axis=1)  # Örnek olarak tarih sütunları çıkarıldı
target = data['OUP']

# Karşılıklı bilgi (mutual information) hesaplama
mutual_info = mutual_info_regression(features, target)

# Her özelliğin karşılıklı bilgi değerlerini içeren bir DataFrame oluşturma
mi_df = pd.DataFrame({'Feature': features.columns, 'Mutual_Info': mutual_info})

# Çubuk grafiği oluşturma
plt.figure(figsize=(10, 6))
sns.barplot(x='Mutual_Info', y='Feature', data=mi_df.sort_values(by='Mutual_Info', ascending=False))
plt.title('Karşılıklı Bilgi Değerleri')
plt.xlabel('Karşılıklı Bilgi')
plt.ylabel('Özellikler')
plt.show()

SARIMAX

In [ ]:
file_path = 'organised_data_final.xlsx'
data = pd.read_excel(file_path)

columns_to_drop = ['MXT', 'MNT']
data = data.drop(columns_to_drop, axis=1)

columns_with_missing_values = ['WDS', 'TMP', 'PRC', 'PRS', 'RLH', 'SSI', 'GSR', 'OUP']

for column in columns_with_missing_values:
    missing_values_indices = data[data[column].isnull()].index
    
    train_data = data[~data[column].isnull()]
    model = SARIMAX(train_data[column], order=(2, 1, 2), seasonal_order=(2, 0, 1, 24))  # SARIMAX modeli örneği
    
    fitted_model = model.fit()
    
    predicted_values = fitted_model.predict(start=missing_values_indices[0], end=missing_values_indices[-1])
    
    data.loc[missing_values_indices, column] = predicted_values

output_file = 'data1.xlsx'
data.to_excel(output_file, index=False)

file_path = 'data1.xlsx'
data2 = pd.read_excel(file_path)

data2['RLH'] = data2['RLH'].astype(int)

data2['DATE'] = pd.to_datetime(data2[['YEAR', 'MONTH', 'DAY']])
data2['DATE'] = data2['DATE'].dt.date

ordered_columns = ['DATE', 'HOUR', 'WDS', 'TMP', 'PRC', 'PRS', 'RLH', 'SSI', 'GSR', 'OUP']
data2 = data2[ordered_columns]

output_file_updated = 'data1.xlsx'
data2.to_excel(output_file_updated, index=False)


XGBREGRESSOR EN İYİ PARAMETRE VE SKOR

In [ ]:
# Özellikler ve hedef değişkeni seç
features = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'WDS', 'MXT', 'MNT', 'TMP', 'PRC', 'PRS', 'RLH', 'SSI', 'GSR']
target = 'OUP'

# Eksik değerleri içermeyen veriyi seç ve eğitim/test setlerine ayır
data_without_missing = data.dropna(subset=features + [target])
train, test = train_test_split(data_without_missing, test_size=0.3, random_state=42)

# Eğitim setini ve test setini eksik verileri doldurmadan kullan
train_features = train[features]
train_target = train[target]

# XGBoost modelini tanımla
xgb = XGBRegressor()

# Grid Search için parametre grid'i oluştur
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.05, 0.1, 0.2]
}

# Grid Search ile en iyi parametreleri bul
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring='neg_mean_squared_error', verbose=2)
grid_search.fit(train_features, train_target)

# En iyi parametreleri ve skorları göster
print("En iyi parametreler:", grid_search.best_params_)
print("En iyi skor:", grid_search.best_score_)

XGBREGRESSOR MODEL EĞİTİMİ

In [ ]:
data = pd.read_excel(file_path)

# Eksik değerlerin olduğu sütunlar
columns_with_missing_values = ['WDS', 'MXT', 'MNT', 'TMP', 'PRC', 'PRS', 'RLH', 'SSI', 'GSR', 'OUP']

# Eksik değerlerin bulunduğu satırların indeksleri
rows_with_missing_values = data[columns_with_missing_values].isnull().any(axis=1)
data_with_missing_values = data[rows_with_missing_values]
data_without_missing_values = data.drop(columns_with_missing_values, axis=1)

# XGBRegressor için en iyi parametreler
best_params = {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 100}

# Imputer tanımla
imputer = SimpleImputer(strategy='mean')

for column in columns_with_missing_values:
    features = data_without_missing_values
    target = data[column]

    # Model oluştur
    model = XGBRegressor(**best_params)

    # Eksik değerleri tahmin et
    imputed_features = imputer.fit_transform(features)
    model.fit(imputed_features[~rows_with_missing_values], target[~rows_with_missing_values])

    missing_values_indices = data_with_missing_values[data_with_missing_values[column].isnull()].index
    missing_values_features = data_without_missing_values.iloc[missing_values_indices]
    predicted_values = model.predict(imputer.transform(missing_values_features))

    # Eksik değerleri doldur
    data.loc[missing_values_indices, column] = predicted_values

# Yeni veri setini kaydet
output_file = 'data2.xlsx'
data.to_excel(output_file, index=False)

file_path = 'data2.xlsx'
data2 = pd.read_excel(file_path)

data2['RLH'] = data2['RLH'].astype(int)

data2['DATE'] = pd.to_datetime(data2[['YEAR', 'MONTH', 'DAY']])
data2['DATE'] = data2['DATE'].dt.date

ordered_columns = ['DATE', 'HOUR', 'WDS', 'TMP', 'PRC', 'PRS', 'RLH', 'SSI', 'GSR', 'OUP']
data2 = data2[ordered_columns]

output_file_updated = 'data2.xlsx'
data2.to_excel(output_file_updated, index=False)


DECISION TREE MODEL EĞİTİMİ

In [ ]:
file_path = 'organised_data_final.xlsx'
data = pd.read_excel(file_path)

columns_with_missing_values = ['WDS', 'MXT', 'MNT', 'TMP', 'PRC', 'PRS', 'RLH', 'SSI', 'GSR', 'OUP']

data_without_missing_values = data.drop(columns_with_missing_values, axis=1)

imputer = SimpleImputer(strategy='mean')

# En iyi parametreleri bulmak için GridSearchCV kullanarak DecisionTreeRegressor'ı ayarlayın
param_grid = {
    'max_depth': [10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

best_params = {}
for column in columns_with_missing_values:
    features = data_without_missing_values
    target = data[column]

    # Eksik değerlerin olduğu satırları belirleyin
    rows_with_missing_values = target.isnull()

    # Eğitim için en iyi parametreleri bulun
    grid_search = GridSearchCV(DecisionTreeRegressor(), param_grid, cv=5)
    grid_search.fit(features[~rows_with_missing_values], target[~rows_with_missing_values])
    best_params[column] = grid_search.best_params_

# En iyi parametrelerle modelleri eğitin ve eksik değerleri doldurun
for column in columns_with_missing_values:
    features = data_without_missing_values
    target = data[column]

    rows_with_missing_values = target.isnull()

    model = DecisionTreeRegressor(**best_params[column])
    imputed_features = imputer.fit_transform(features)
    model.fit(imputed_features[~rows_with_missing_values], target[~rows_with_missing_values])

    missing_values_indices = target[rows_with_missing_values].index
    missing_values_features = data_without_missing_values.iloc[missing_values_indices]
    predicted_values = model.predict(imputer.transform(missing_values_features))

    data.loc[missing_values_indices, column] = predicted_values

output_file = 'data3.xlsx'
data.to_excel(output_file, index=False)

file_path = 'data3.xlsx'
data2 = pd.read_excel(file_path)

data2['RLH'] = data2['RLH'].astype(int)

data2['DATE'] = pd.to_datetime(data2[['YEAR', 'MONTH', 'DAY']])
data2['DATE'] = data2['DATE'].dt.date

ordered_columns = ['DATE', 'HOUR', 'WDS', 'TMP', 'PRC', 'PRS', 'RLH', 'SSI', 'GSR', 'OUP']
data2 = data2[ordered_columns]

output_file_updated = 'data3.xlsx'
data2.to_excel(output_file_updated, index=False)

1. İSTATİSTİK

In [ ]:
# Veri setini oku
df = pd.read_excel('data3.xlsx')
df = df.dropna()

# 'DATE' sütununu DateTime formatına dönüştürme ve 'Month' sütununu oluşturma
df['DATE'] = pd.to_datetime(df['DATE'])
df['Month'] = df['DATE'].dt.month

# Görselleştirme için verileri hazırlama
nem_prc_oranlari = {}

nem_araliklari = [(i, i + 10) for i in range(0, 100, 10)]

nem_oranlari_list = []  # DataFrame'e dönüştürmek için veri listesi

for nem_araligi in nem_araliklari:
    alt_sinir, ust_sinir = nem_araligi
    filt_nem = (df['RLH'] >= alt_sinir) & (df['RLH'] < ust_sinir)
    for ay in range(1, 13):
        filt_ay = df['Month'] == ay
        filtered_df = df[filt_nem & filt_ay]

        yağış_oranı = (filtered_df['PRC'] > 0).mean() * 100
        yağışlı_df = filtered_df[filtered_df['PRC'] > 0]
        ortalama_prc = yağışlı_df['PRC'].mean() if not yağışlı_df.empty else 0
        toplam_saat_sayısı = filtered_df.shape[0]
        ortalama_oup = filtered_df['OUP'].mean() if not filtered_df.empty else 0

        if not pd.isnull(yağış_oranı) or toplam_saat_sayısı == 0:
            nem_oranlari_list.append({'Nem_Araligi': f"{nem_araligi[0]}-{nem_araligi[1]}", 'Month': ay,
                                      'Yağış_Oranı': yağış_oranı,
                                      'Toplam_Saat': toplam_saat_sayısı,
                                      'Ortalama_PRC': ortalama_prc,
                                      'Ortalama_OUP': ortalama_oup})

# Liste üzerinden DataFrame oluşturma
nem_oranlari_df = pd.DataFrame(nem_oranlari_list)   

# Çizgi grafiği oluşturma
plt.figure(figsize=(12, 8))
sns.lineplot(x='Month', y='Yağış_Oranı', hue='Nem_Araligi', data=nem_oranlari_df)
plt.title('Aylara Göre Nem Aralıklarına Göre Yağış Oranı')
plt.xlabel('Ay')
plt.ylabel('Yağış Oranı (%)')
plt.legend(title='Nem Aralığı')
plt.show()

2. İSTATİSTİK

In [ ]:
# Veri setini oku ve gereksiz sütunları kaldır
df = pd.read_excel('data3.xlsx').dropna()

# 'DATE' sütununu datetime olarak işle ve 'Month' sütununu oluştur
df['DATE'] = pd.to_datetime(df['DATE'])
df['Month'] = df['DATE'].dt.month

# Sadece OUP değeri sıfır olmayan saatleri içeren bir veri seti oluştur
non_zero_oup_hours = df[df['OUP'] != 0]

# Ay ve saate göre OUP'nin ortalama değerlerini grupla
oup_by_hour_and_month = non_zero_oup_hours.groupby(['Month', 'HOUR'])['OUP'].mean().reset_index()

# Ay isimlerini Türkçe olarak tanımla
months_dict = {
    1: 'Ocak',
    2: 'Şubat',
    3: 'Mart',
    4: 'Nisan',
    5: 'Mayıs',
    6: 'Haziran',
    7: 'Temmuz',
    8: 'Ağustos',
    9: 'Eylül',
    10: 'Ekim',
    11: 'Kasım',
    12: 'Aralık'
}

# Ay isimleri ve saatlere göre üretim ortalamalarını içeren bir dosyaya yaz
with open('saatlikuretim.txt', 'w') as file:
    for month in oup_by_hour_and_month['Month'].unique():
        file.write(f"{months_dict[month]} ayı saatlere göre üretim ortalaması\n")
        for index, row in oup_by_hour_and_month[oup_by_hour_and_month['Month'] == month].iterrows():
            file.write(f"Saat: {row['HOUR']} - Ortalama OUP: {row['OUP']:.2f}\n")
        file.write("\n")

# Çubuk grafikler oluşturma
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(12, 10))

for i, (month, data) in enumerate(oup_by_hour_and_month.groupby('Month')):
    row = i // 4
    col = i % 4
    ax = axes[row, col]

    ax.bar(data['HOUR'], data['OUP'], align='center')
    ax.set_title(months_dict[month])
    ax.set_ylabel('Ortalama OUP')
    ax.set_xticks(data['HOUR'])

plt.tight_layout()
plt.show()

3. İSTATİSTİK

In [ ]:
# Veri setini oku
df = pd.read_excel('data3.xlsx')

# Rüzgar hızı aralıklarını belirle
wds_intervals = [(i, i + 1) for i in range(0, 10)]

# Rüzgar hızı aralıklarına göre ortalama üretim değerlerini hesapla
wds_oup_means = {}
for interval in wds_intervals:
    lower, upper = interval
    filtered_df = df[(df['WDS'] >= lower) & (df['WDS'] < upper)]
    oup_mean = filtered_df['OUP'].mean()
    wds_oup_means[interval] = oup_mean

# Ortalama üretim değerlerini metin dosyasına yaz
with open('rüzgar_oranlari.txt', 'w') as file:
    file.write("Rüzgar Hızı Aralıklarına Göre Ortalama Üretim Değerleri:\n")
    for interval, oup_mean in wds_oup_means.items():
        file.write(f"{interval[0]}-{interval[1]} aralığında WDS için Ortalama OUP: {oup_mean:.2f}\n")

# Pasta grafiği oluşturma
labels = [f"{interval[0]}-{interval[1]}" for interval in wds_oup_means.keys()]
values = list(wds_oup_means.values())

plt.figure(figsize=(8, 8))
plt.pie(values, labels=labels, autopct='%1.1f%%')
plt.title('Rüzgar Hızı Aralıklarına Göre Ortalama Üretim Değerleri')
plt.show()

4. İSTATİSTİK

In [ ]:
df = pd.read_excel('data3.xlsx')

temperature_intervals = [(i, i + 3) for i in range(-21, 37, 3)]

temperature_oup_means = {}
for interval in temperature_intervals:
    lower, upper = interval
    filtered_df = df[(df['TMP'] >= lower) & (df['TMP'] < upper)]
    oup_mean = filtered_df['OUP'].mean()
    temperature_oup_means[interval] = oup_mean

with open('sicaklik_oranlari.txt', 'w') as file:
    file.write("Sıcaklık Aralıklarına Göre Ortalama Üretim Değerleri:\n")
    for interval, oup_mean in temperature_oup_means.items():
        file.write(f"{interval[0]}-{interval[1]} aralığında Sıcaklık için Ortalama OUP: {oup_mean:.2f}\n")

5. İSTATİSTİK

In [ ]:
df = pd.read_excel('data3.xlsx')

gsr_bins = np.linspace(830, 860, 11)  # 0-1000 aralığını 100'erli böler
oup_means = df.groupby(pd.cut(df['PRS'], gsr_bins)).mean()['OUP']

plt.figure(figsize=(10, 6))
plt.plot(oup_means.index.astype(str), oup_means.values, marker='o')  # .astype(str) kullanarak kategorik indeksleri dizeye çevir
plt.title('PRS Aralıklarına Göre Ortalama OUP Değeri')
plt.xlabel('PRS Aralığı')
plt.ylabel('Ortalama OUP Değeri')
plt.grid(True)
plt.show()

ÖZELLİK SKORLARIYLA BÜTÜN MODELLERI KIYASLAMA

In [ ]:
file_path = 'organised_data_final.xlsx'
data2 = pd.read_excel(file_path)

# Veri setinin temel istatistiksel özetini almak
summary_stats = data2.describe()
summary_stats.to_excel('summary_stats.xlsx')

#####

file_path = 'data1.xlsx'
data2 = pd.read_excel(file_path)

# Veri setinin temel istatistiksel özetini almak
summary_stats = data2.describe()
summary_stats.to_excel('summary_stats2.xlsx')

#####

file_path = 'data2.xlsx'
data2 = pd.read_excel(file_path)

# Veri setinin temel istatistiksel özetini almak
summary_stats = data2.describe()
summary_stats.to_excel('summary_stats3.xlsx')

#####

file_path = 'data3.xlsx'
data2 = pd.read_excel(file_path)

# Veri setinin temel istatistiksel özetini almak
summary_stats = data2.describe()
summary_stats.to_excel('summary_stats4.xlsx')